In [3]:
import torch
import gc
from transformers import AutoTokenizer, AutoModelForCausalLM, MistralForCausalLM
from peft import AutoPeftModelForCausalLM

gc.collect()
torch.cuda.empty_cache()
model_directory = "/home/ksaff/Desktop/ttyd/api/model"
tokenizer = AutoTokenizer.from_pretrained(model_directory)
model = AutoPeftModelForCausalLM.from_pretrained(model_directory,
                                         load_in_8bit=True,
                                         device_map={'': 0})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


IT WORKS
['/home/ksaff/Desktop/ttyd/api/Mistral-7B-v0.1/pytorch_model-00001-of-00002.bin', '/home/ksaff/Desktop/ttyd/api/Mistral-7B-v0.1/pytorch_model-00002-of-00002.bin']


Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.93s/it]


In [4]:
ddl = """CREATE TABLE Customers (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    email TEXT,
    DOB DATE
);

CREATE TABLE Orders (
    orderID INTEGER PRIMARY KEY AUTOINCREMENT,
    customerID INTEGER,
    productID INTEGER,
    quantity INTEGER,
    orderDate TIMESTAMP
);

CREATE TABLE Products (
    productID INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    category TEXT,
    price DECIMAL(10, 2),
    inStock INTEGER
);

CREATE TABLE Employees (
    empID INTEGER PRIMARY KEY AUTOINCREMENT,
    firstName TEXT,
    lastName TEXT,
    position TEXT,
    salary INTEGER
);

CREATE TABLE Inventory (
    productID INTEGER,
    location TEXT,
    quantity INTEGER
);"""

In [5]:
import os

input = ddl
prompt = 'Make SQLite query based on DDL and instruction.'
instruction = "What are the distinct positions in company?"
text = (
    prompt
    + '### Instruction:\n'
    + instruction
    + '### Input:\n'
    + input
    + '### Response:\n'
)

In [6]:
input_ids = tokenizer.encode(text, return_tensors='pt')
input_ids = input_ids.to('cuda')

In [17]:
inputs = {
    'input_ids': input_ids,
    'max_length': 1000
}

output = model.generate(**inputs)

In [19]:
import re

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

match = re.search(r'### Response:\s*(.*)', generated_text)
response = match.group(1)
response

'SELECT DISTINCT position FROM Employees;'

In [20]:
import sqlite3
from dotenv import load_dotenv
load_dotenv()

conn = sqlite3.connect('/home/ksaff/Desktop/ttyd/ComplexDB.db')
c = conn.cursor()

def execute_query(query):

    c.execute(query)
    result = c.fetchall()

    # Extract column names
    columns = [desc[0] for desc in c.description]
    
    # Prepare the response
    response = {"query": query, "headers": columns, "data": result}

    return response

In [17]:
result = execute_query(response)
print(f'User instruction: {instruction}')
print(f'model output: {response}')
'Result query:', result['data']

User instruction: What are the distinct positions in company?
model output: SELECT DISTINCT position FROM Employees;


('Result query:',
 [('Sales',), ('Designer',), ('HR',), ('Developer',), ('Manager',)])